In [7]:
from scenariogeneration import xosc

In [8]:
# Fixed parameters
DRONE_ID = "5"
DRONE_CATALOG_NAME = "camera_drone"

CATALOG_PATH = "../Catalogs/Vehicles"
CATALOG_NAME = "VehicleCatalog"

OPENDRIVE_PATH = "../odr/GaragePlanSouth.xodr"
SCENARIO_NAME = "DroneScenario"
SCENARIO_FILE_NAME = SCENARIO_NAME + ".xosc"

In [9]:
# Dynamic parameters
drone_starting_point = xosc.WorldPosition(0,0,5,0,0,0)
drone_speed = 10 / 3.6 # m/s
drone_acceleration = 2 # m/s2

In [10]:
# Preamble
init = xosc.Init()
osc_params = xosc.ParameterDeclarations()
osc_catalogs = xosc.Catalog()
osc_catalogs.add_catalog(CATALOG_NAME, CATALOG_PATH)
osc_road_network = xosc.RoadNetwork(roadfile=OPENDRIVE_PATH)
osc_entities = xosc.Entities()
osc_storyboard = xosc.StoryBoard(init)
osc_act = xosc.Act("start")

drone_maneuver_group = xosc.ManeuverGroup(DRONE_ID + ",maneuver_group")
drone_maneuver = xosc.Maneuver(DRONE_ID + ",maneuver")
drone_maneuver_group.add_actor(DRONE_ID)

In [11]:
# Starting position
osc_entities = osc_entities.add_scenario_object(DRONE_ID, xosc.CatalogReference(CATALOG_NAME, DRONE_CATALOG_NAME))
drone_start_teleport = xosc.TeleportAction(drone_starting_point)
drone_set_speed = xosc.AbsoluteSpeedAction(
	drone_speed,
	xosc.TransitionDynamics(
		xosc.DynamicsShapes.linear,
		xosc.DynamicsDimension.rate,
		drone_acceleration
	)
)
init.add_init_action(DRONE_ID, drone_start_teleport)
init.add_init_action(DRONE_ID, drone_set_speed)

In [12]:
# Drone movement profile

# Start moving
times = [2.5, 5, 7.5, 10, 12.5, 15]
positions = [
	xosc.WorldPosition(0,0,5,0,0,0),
	xosc.WorldPosition(1,-2,5,0,0,0),
	xosc.WorldPosition(2,-4,5,0,0,0),
	xosc.WorldPosition(3,-6,5,0,0,0),
	xosc.WorldPosition(4,-8,5,0,0,0),
	xosc.WorldPosition(5,-10,5,0,0,0)
]

polyline = xosc.Polyline(times, positions)
trajectory = xosc.Trajectory(DRONE_ID + ",init_follow_trajectory", closed=False)
trajectory.add_shape(polyline)
drone_follow_trajectory = xosc.FollowTrajectoryAction(
	trajectory,
	xosc.FollowMode.position,
	xosc.ReferenceContext.relative,
	1,
	0
)
drone_scenario_started = xosc.ValueTrigger(
    DRONE_ID + ",scenario_started",
    0,
    xosc.ConditionEdge.none,
    xosc.SimulationTimeCondition(0, xosc.Rule.greaterThan)
)
drone_follow_trajectory_event = xosc.Event(
	DRONE_ID + ",drone_follow_trajectory_event",
	xosc.Priority.parallel
)
drone_follow_trajectory_event.add_trigger(drone_scenario_started)
drone_follow_trajectory_event.add_action(DRONE_ID + ",init_follow_trajectory", drone_follow_trajectory)
drone_maneuver.add_event(drone_follow_trajectory_event)

# Stop
drone_brake = xosc.AbsoluteSpeedAction(
	0,
	xosc.TransitionDynamics(
		xosc.DynamicsShapes.linear,
		xosc.DynamicsDimension.time,
		-drone_acceleration
	)
)
drone_reach_brake_position = xosc.EntityTrigger(
    name=DRONE_ID + ",reach_brake_position",
    delay=0,
    conditionedge=xosc.ConditionEdge.rising,
    entitycondition=xosc.ReachPositionCondition(positions[-1], 1.0), # tolerance
    triggerentity=DRONE_ID
)
drone_brake_event = xosc.Event(
	DRONE_ID + ",stop_event",
    xosc.Priority.overwrite,
)

drone_brake_event.add_trigger(drone_reach_brake_position)
drone_brake_event.add_action(DRONE_ID + ",brake_to_stop", drone_brake)
drone_maneuver.add_event(drone_brake_event)

In [13]:
# Collect into a scenario and write to file
drone_maneuver_group.add_maneuver(drone_maneuver)
osc_act.add_maneuver_group(drone_maneuver_group)
osc_storyboard.add_act(osc_act)
osc_file = xosc.Scenario(
		name=SCENARIO_NAME,
		author="AstaZero 1.0",
		parameters=osc_params,
		entities=osc_entities,
		storyboard=osc_storyboard,
		roadnetwork=osc_road_network,
		catalog=osc_catalogs
	)

osc_file.write_xml(SCENARIO_FILE_NAME)